# 4. Matrix Factorization 기반 추천
  - 알고리즘 
    - 메모리 기반 - collaborative filtering
    - 모델 기반 - matrix factorization

## 4.1 Matrix Factorization 방식의 원리
 - 행렬 요인화 - 사용자x아이템으로 구성된 하나의 행렬을 2개의 행렬로 분해하는 방법
 - R : rating matrix, P : User latent matrix, Q : item latent matrix
 - R을 사용자 행렬 P와 아이템 행렬 Q로 쪼개어 분석하는 것
 - $R \approx P \times Q^T = \hat{R}$을 구하는 것
  - 이 때 $\hat{R}$은 $R$의 예측치이며 $R$과 비슷하게 되도록하는 $P$와 $Q$를 구하는 것이 목표
  - $P$와 $Q$에는 $K$개의 요인의 값으로 행렬이 이루어져 있는데 $P$에서는 $K$개의 사용자의 특성을, $Q$에서는 $K$개의 아이템의 특성을 나타냄, 그리고 이를 잠재요인(latent factor)라고 부름

## 4.2 SGD(Stochastic Gradient Decent)를 사용한 MF 알고리즘
 - 알고리즘
  1. 잠재요인 $K$의 개수 선정
  2. 주어진 $K$에 따라 행렬 $P$,$Q$ 초기화
  3. 예측 평점을 구함
  4. $R$에 있는 실제 평점과 $\hat{R}$의 예측 평점의 오차를 줄이기 위해 $P$,$Q$ 값 수정 -> SGE 방법 사용
  5. 전체오차를 줄이기 위해 iteration 값 혹은 목표치에 근접할 때까지 3,4번을 반복
 - 수식 생략
 - 고려사항
  1. 과적합 방지를 위해 정규화(regularization)항 추가
  2. 사용자와 아이템의 경향성 -> 평가가 높거나 낮거나 할 수 있음

## 4.3 SGD를 사용한 MF 기본 알고리즘
 - class 활용한 코딩 진행

In [31]:
import numpy as np
import pandas as pd
from utility import *
from sklearn.utils import shuffle
import matplotlib.pyplot as plt

In [6]:
_ ,_ , ratings = getData()
ratings.reset_index(inplace=True)
ratings = ratings.drop('timestamp',axis=1)
ratings = ratings[['user_id','movie_id','rating']].astype(int)

In [10]:
# MF class
class MF:
    def __init__(self, ratings, K, alpha, beta, iterations, verbose=True):
        self.R = np.array(ratings.fillna(0))
        self.num_users, self.num_items = np.shape(self.R)
        self.K = K
        self.alpha = alpha
        self.beta = beta
        self.iterations = iterations
        self.verbose = verbose
    
    # RMSE 계산
    def rmse(self):
        xs, ys = self.R.nonzero() # 0이 아닌 인덱스 리턴
        self.predictions = []
        self.errors = []
        for x,y in zip(xs, ys):
            prediction = self.get_prediction(x,y)
            self.predictions.append(prediction)
            self.errors.append(self.R[x,y] - prediction)
        self.predictions = np.array(self.predictions)
        self.errors = np.array(self.errors)
        return np.sqrt(np.mean(self.errors**2))
    
    def train(self):
        # P행렬과 Q행렬 초기화
        self.P = np.random.normal(scale=1./self.K, size=(self.num_users, self.K)) # (num_users, K)
        self.Q = np.random.normal(scale=1./self.K, size=(self.num_items, self.K)) # (num_items, K)
        self.b_u = np.zeros(self.num_users) # (num_users,)
        self.b_d = np.zeros(self.num_items) # (num_items,)
        self.b = np.mean(self.R[self.R.nonzero()]) # 전체 평균
        rows, columns = self.R.nonzero()
        self.samples = [(i,j,self.R[i,j]) for i,j in zip(rows,columns)]
        
        training_process = []
        for i in range(self.iterations):
            np.random.shuffle(self.samples)
            self.sgd()
            rmse = self.rmse()
            training_process.append((i+1,rmse))
            if self.verbose:
                if (i+1)%10 ==0:
                    print(f'iteration : {i+1} ; Train RMSE = {rmse}')
        return training_process
    
    # 예측값 계산
    def get_prediction(self, i, j):
        prediction = self.b + self.b_u[i] + self.b_d[j] + self.P[i,:].dot(self.Q[j,:].T)
        return prediction
    
    # stochastic gradient descent 
    def sgd(self):
        for i, j, r in self.samples:
            prediction = self.get_prediction(i,j)
            e = r-prediction
            
            self.b_u[i] += self.alpha*(e - self.beta*self.b_u[i])
            self.b_d[j] += self.alpha*(e - self.beta*self.b_d[j])
            
            self.P[i,:] += self.alpha*(e*self.Q[j,:] - self.beta*self.P[i,:])
            self.Q[j,:] += self.alpha*(e*self.P[i,:] - self.beta*self.Q[j,:])

In [11]:
R_temp = ratings.pivot(index='user_id', columns='movie_id', values = 'rating')
mf = MF(R_temp, 30, 0.001, 0.02, 100, True)
train_process = mf.train()

iteration : 10 ; Train RMSE = 0.9584980212318277
iteration : 20 ; Train RMSE = 0.9373561041328844
iteration : 30 ; Train RMSE = 0.9280656868205587
iteration : 40 ; Train RMSE = 0.9225230970275939
iteration : 50 ; Train RMSE = 0.9183510357036374
iteration : 60 ; Train RMSE = 0.9143836352410551
iteration : 70 ; Train RMSE = 0.9096480759600305
iteration : 80 ; Train RMSE = 0.903154973886412
iteration : 90 ; Train RMSE = 0.8941048495610546
iteration : 100 ; Train RMSE = 0.8825226802288026


## 4.4 train/test 분리 MF 알고리즘
 - shuffle을 사용한 train_test split 방법 사용

In [13]:
TRAIN_SIZE = 0.75
ratings = shuffle(ratings, random_state = 1)
cutoff = int(TRAIN_SIZE*len(ratings))
ratings_train = ratings.iloc[:cutoff]
ratings_test = ratings.iloc[cutoff:]

In [28]:
class NEW_MF:
    def __init__(self, ratings, K, alpha, beta, iterations, verbose = True):
        self.R = np.array(ratings.fillna(0))
        self.K = K
        self.num_users, self.num_items = self.R.shape
        self.alpha = alpha
        self.beta = beta
        self.iterations = iterations
        self.verbose = verbose
        # user_id, item_id를 R의 index와 매핑하기 위한 dictinary 생성
        item_id_index = []
        index_item_id = []
        for i, one_id in enumerate(ratings):
            item_id_index.append([one_id, i])
            index_item_id.append([i, one_id])
        self.item_id_index = dict(item_id_index)
        self.index_item_id = dict(index_item_id)
        user_id_index = []
        index_user_id = []
        for i, one_id in enumerate(ratings.index):
            user_id_index.append([one_id, i])
            index_user_id.append([i, one_id])
        self.user_id_index = dict(user_id_index)
        self.index_item_id = dict(index_user_id)
    
    # RMSE 계산
    def rmse(self):
        xs, ys = self.R.nonzero() # 0이 아닌 인덱스 리턴
        self.predictions = []
        self.errors = []
        for x,y in zip(xs, ys):
            prediction = self.get_prediction(x,y)
            self.predictions.append(prediction)
            self.errors.append(self.R[x,y] - prediction)
        self.predictions = np.array(self.predictions)
        self.errors = np.array(self.errors)
        return np.sqrt(np.mean(self.errors**2))
    
    def train(self):
        # P행렬과 Q행렬 초기화
        self.P = np.random.normal(scale=1./self.K, size=(self.num_users, self.K)) # (num_users, K)
        self.Q = np.random.normal(scale=1./self.K, size=(self.num_items, self.K)) # (num_items, K)
        self.b_u = np.zeros(self.num_users) # (num_users,)
        self.b_d = np.zeros(self.num_items) # (num_items,)
        self.b = np.mean(self.R[self.R.nonzero()]) # 전체 평균
        rows, columns = self.R.nonzero()
        self.samples = [(i,j,self.R[i,j]) for i,j in zip(rows,columns)]
        
        training_process = []
        for i in range(self.iterations):
            np.random.shuffle(self.samples)
            self.sgd()
            rmse = self.rmse()
            training_process.append((i+1,rmse))
            if self.verbose:
                if (i+1)%10 ==0:
                    print(f'iteration : {i+1} ; Train RMSE = {rmse}')
        return training_process
    
    # 예측값 계산
    def get_prediction(self, i, j):
        prediction = self.b + self.b_u[i] + self.b_d[j] + self.P[i,:].dot(self.Q[j,:].T)
        return prediction
    
    # stochastic gradient descent 
    def sgd(self):
        for i, j, r in self.samples:
            prediction = self.get_prediction(i,j)
            e = r-prediction
            
            self.b_u[i] += self.alpha*(e - self.beta*self.b_u[i])
            self.b_d[j] += self.alpha*(e - self.beta*self.b_d[j])
            
            self.P[i,:] += self.alpha*(e*self.Q[j,:] - self.beta*self.P[i,:])
            self.Q[j,:] += self.alpha*(e*self.P[i,:] - self.beta*self.Q[j,:])
            
    # test set 선정
    def set_test(self, ratings_test):
        test_set = []
        for i in range(len(ratings_test)):
            x = self.user_id_index[ratings_test.iloc[i,0]]
            y = self.item_id_index[ratings_test.iloc[i,1]]
            z = ratings_test.iloc[i,2]
            test_set.append([x,y,z])
            self.R[x,y] = 0
        self.test_set = test_set
        return test_set
    
    def test_rmse(self):
        error = 0
        for one_set in self.test_set:
            predicted = self.get_prediction(one_set[0],one_set[1])
            error += pow(one_set[2] - predicted, 2)
        return np.sqrt(error/len(self.test_set))
    
    def test(self):
        self.P = np.random.normal(scale=1./self.K, size=(self.num_users, self.K)) # (num_users, K)
        self.Q = np.random.normal(scale=1./self.K, size=(self.num_items, self.K)) # (num_items, K)
        self.b_u = np.zeros(self.num_users) # (num_users,)
        self.b_d = np.zeros(self.num_items) # (num_items,)
        self.b = np.mean(self.R[self.R.nonzero()]) # 전체 평균
        
        rows, columns = self.R.nonzero()
        self.samples = [(i,j,self.R[i,j]) for i,j in zip(rows,columns)]
        
        training_process = []
        for i in range(self.iterations):
            np.random.shuffle(self.samples)
            self.sgd()
            rmse = self.rmse()
            rmse_test = self.test_rmse()
            training_process.append((i+1,rmse,rmse_test))
            if self.verbose:
                if (i+1)%10 ==0:
                    print(f'iteration : {i+1} ; Train RMSE = {rmse} ; Test RMSE = {rmse_test:.4f}')
        return training_process
    
    def get_one_prediction(self, user_id, item_id):
        return self.get_prediction(self.user_id_index[user_id], self.item_id_index[item_id])
    
    def full_prediction(self):
        return self.b + self.b_u[:,np.newaxis] + self.b_d[np.newaxis,:] + self.P.dot(self.Q.T)

In [29]:
mf = NEW_MF(R_temp, 30, 0.001, 0.02, 100, True)
test_set = mf.set_test(ratings_test)
result = mf.test()

iteration : 10 ; Train RMSE = 0.9659127342419507 ; Test RMSE = 0.9834
iteration : 20 ; Train RMSE = 0.9409755423537928 ; Test RMSE = 0.9645
iteration : 30 ; Train RMSE = 0.9297902938799715 ; Test RMSE = 0.9567
iteration : 40 ; Train RMSE = 0.9230921523384817 ; Test RMSE = 0.9524
iteration : 50 ; Train RMSE = 0.9183722245164371 ; Test RMSE = 0.9497
iteration : 60 ; Train RMSE = 0.9145280396083484 ; Test RMSE = 0.9479
iteration : 70 ; Train RMSE = 0.9109106173801017 ; Test RMSE = 0.9464
iteration : 80 ; Train RMSE = 0.9069940853822434 ; Test RMSE = 0.9452
iteration : 90 ; Train RMSE = 0.9022604065251304 ; Test RMSE = 0.9438
iteration : 100 ; Train RMSE = 0.8961293023097657 ; Test RMSE = 0.9421


## 4.5 MF의 최적 파라미터 찾기
 - K가 커지면 over-fitting 발생
 - 최적의 iterations 및 K 찾기
   1. 넓은 범위(50~260)에 대해 10간격으로 정확도 계산
   2. 적절한 K에 대해 앞뒤 10정도의 범위에서 1의 간격으로 최적의 K값 확인
   3. iteration은 충분히 큰 숫자를 통해 RMSE 변화를 관찰, 그 후 iteration 확인

In [30]:
# 최적의 K 구하기
results = []
index = []
for K in range(50,261,10):
    print(f'K = {K}')
    mf = NEW_MF(R_temp, K, 0.001, 0.02, 300, True)
    test_set = mf.set_test(ratings_test)
    result = mf.test()
    index.append(K)
    results.append(result)

K = 50
iteration : 10 ; Train RMSE = 0.9661141952422679 ; Test RMSE = 0.9834
iteration : 20 ; Train RMSE = 0.9414200596935819 ; Test RMSE = 0.9644
iteration : 30 ; Train RMSE = 0.9304779508966711 ; Test RMSE = 0.9566
iteration : 40 ; Train RMSE = 0.9240709602016239 ; Test RMSE = 0.9523
iteration : 50 ; Train RMSE = 0.9197038880057534 ; Test RMSE = 0.9497
iteration : 60 ; Train RMSE = 0.916321739855097 ; Test RMSE = 0.9478
iteration : 70 ; Train RMSE = 0.9133089060051923 ; Test RMSE = 0.9465
iteration : 80 ; Train RMSE = 0.9102149333664625 ; Test RMSE = 0.9453
iteration : 90 ; Train RMSE = 0.9065776791366428 ; Test RMSE = 0.9441
iteration : 100 ; Train RMSE = 0.9018738128879455 ; Test RMSE = 0.9426
iteration : 110 ; Train RMSE = 0.895501816508779 ; Test RMSE = 0.9405
iteration : 120 ; Train RMSE = 0.8868886554709896 ; Test RMSE = 0.9378
iteration : 130 ; Train RMSE = 0.875760644911509 ; Test RMSE = 0.9344
iteration : 140 ; Train RMSE = 0.8623222944187282 ; Test RMSE = 0.9307
iteration :

KeyboardInterrupt: 

In [ ]:
# 최적의 iterations 구하기
summary = []
for i in range(len(results)):
    RMSE = []
    for result in results[i]:
        RMSE.append(result[2])
    min = np.min(RMSE)
    j = RMSE.index(min)
    summary.append([index[i],j+1, RMSE[j]])

In [ ]:
# 그래프 그리기
plt.plot(index,[x[2] for x in summary])
plt.ylim(0.89, 0.94)
plt.xlabel('K')
plt.ylabel('RMSE')
plt.show()

## 4.7 MF와 SVD
 - SVD(singular value decomposition) - 3개의 행렬로 분해, 이를 원래의 행렬로 재현시키는 방법
   - (mxn) = (mxm)x(mxn)x(nxn)
   - null 값에 대한 계산이 되지 않음
     - 이를 0으로 치환했을 때 0으로 재현하려고 하기 때문에 추천시스템에 적합하지 않음
     - 반면 MF의 경우, null 값에 대해 값을 주기 때문에 추천시스템에 사용 가능